## White noise localization
### Extension 2 : average the errors between the two mics

In [ ]:
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt 
import wave 
from scipy.io import wavfile
import soundfile
import time 
from scipy import signal
import logging
import datetime
from itertools import combinations
from Processing import calculate_angle_error


In [ ]:
def extension2(J=1,runs=10,SNR=20,step=1,hrtf=0):
    path_kemar = 'data/kemar_h_theta_1deg_time.npy' #  the path to the file
    path_lego = 'data/lego1_h_theta_time.npy' #  path to lego mics data 

    if hrtf==0:
        hrir = np.random.randn(160,180,6)
    if hrtf==1:
        hrir  = np.load(path_lego)
    if hrtf==2:
        hrir  = np.load(path_kemar) #head related transfer function
        
    T,D,M = hrir.shape

    fs = 16000 #samples/second
    T_ =0.5 #length of signal in time domain 
    N = int(T_*fs) #number of samples of the signal 

    mean = 0
    std = 1 
    num_samples = int(hrir.shape[0])
    hrir_1 = np.zeros((160, 360))
    hrir_2 = np.zeros((160, 360))
    hrir_1 = hrir[:,:,0] 
    hrir_2 = hrir[:,:,1]
    obs_len = N + int(hrir.shape[0]) -1  #length of the convolution

    Df = int(hrir.shape[1]) #directions suppr


    avg_err = 0
    perfect = 0
    start = time.time()
    for rns in range(runs):
        
        St_all = np.zeros((N, J))
        for j in range(J):
            St_all[:, j] = np.random.normal(mean, std, size=N) #white noise

        
        theta = np.random.choice(range(Df), J, replace=False) #choose the directions randomly on the grid   

    
    
        yt = np.zeros((obs_len, )) #recorded time domain signal  voir deuxième paramètre 
        yt2 = np.zeros((obs_len, ))

    
        for j in range(J):
            yt += np.convolve(St_all[:, j], hrir_1[:, theta[j]]) #source signal convolved with corresponding directional response
            yt2 += np.convolve(St_all[:, j], hrir_2[:, theta[j]])
        
        # Generate noise at required SNR    
        sig_norm = np.linalg.norm(yt)
        noise_t = np.random.randn(obs_len, ) #additive gaussian noise
        noise_norm = sig_norm/(10.**(SNR/20.))
        noise_t = noise_norm*noise_t/np.linalg.norm(noise_t)

        yt += noise_t #noisy signal
    
        # noise for second mic
        sig_norm = np.linalg.norm(yt)
        noise_t = np.random.randn(obs_len, ) #additive gaussian noise
        noise_norm = sig_norm/(10.**(SNR/20.))
        noise_t = noise_norm*noise_t/np.linalg.norm(noise_t)

        yt2 += noise_t #noisy signal
    
    
    
        f, t, y = signal.stft(yt, fs=1.0, window='hann', nperseg=160)
        f, t, y2 = signal.stft(yt2, fs=1.0, window='hann', nperseg=160)
    
        N_ = y.shape[1] #number of frames

        y_mean = np.mean(np.abs(y)**2, axis=1) #mean power frame
        y_mean = y_mean/np.linalg.norm(y_mean) #normalize the observation
    
        y_mean2 = np.mean(np.abs(y2)**2, axis=1) #mean power frame
        y_mean2 = y_mean2/np.linalg.norm(y_mean2) #normalize the observation

        H1 = np.fft.rfft(hrir_1,axis=0)
        H_coarse1 = H1[:,::step]
    
        H2 = np.fft.rfft(hrir_2,axis=0)
        H_coarse2 = H2[:,::step]
    
        anglesf = np.arange(Df, dtype=np.int64)*360./Df # list of angles in degrees
        # Initialize variables
    
        D = H_coarse1.shape[1]
    
        best_ind = np.inf #index corresponding to best direction tuple
        smallest_norm = np.inf #smallest projection error
        best_dir = theta #best direction tuple
        
        # Search all combinations
        pairs2 = combinations(range(D), J) #all combinations in range(D) and length J=2
    
    
        for q2, d2 in enumerate(pairs2): 
            Bj1 = np.abs(H_coarse1[:, d2])**2 #vectors in current guess contains the two vectors 
            Pj1 = Bj1.dot(np.linalg.pinv(Bj1)) #projection matrix
            proj_error1 = np.linalg.norm((np.eye(int(y.shape[0])) - Pj1).dot(y_mean)) #projection error1
            
            Bj2 = np.abs(H_coarse2[:, d2])**2 #vectors in current guess contains the two vectors 
            Pj2 = Bj2.dot(np.linalg.pinv(Bj2)) #projection matrix
            proj_error2 = np.linalg.norm((np.eye(int(y.shape[0])) - Pj2).dot(y_mean2)) #projection error2
            
            proj_error = (proj_error1 + proj_error2)/2
        
            if proj_error <= smallest_norm:
                smallest_norm = proj_error
                best_ind = q2
                best_dir = d2
    
        theta_hat = step*np.array(best_dir) #map coarse index to fine index
        min_err, best_perm = calculate_angle_error(theta, theta_hat, anglesf) #calculate error between chosen and true directions        
        #min_err, best_perm = calculate_angle_error(np.round(theta,-1), theta_hat, anglesf)     
        if min_err==0:
            perfect+=1
        
        end = time.time() - start
        avg_err += min_err 
        print('iteration ' + str(rns+1) +' error : '+ str(min_err) + '  ground truth : ' + str(theta)+ '  estimation : ' + str(best_perm) )

    avg_err = avg_err/runs
    print(str(J) +' sources')
    print('average error : ' + str(avg_err) +' over ' + str(runs) + ' runs')
    accuracy = (perfect/runs)*100
    print('accuracy : ' + str(accuracy) + ' %')
    
    return avg_err,accuracy



#### Call the function extension 2 
we fix the parameters : 

J : number of sources to detect   
runs : number of runs over which we calculate the average error   
SNR : signal to noise ration  
hrtf : to choose Lego, Kemar, or random data   

In [ ]:
runs = 100
# example with 2 sources on 100 runs and lego data
avg_err, accuracy = extension2(J=2,runs=runs,SNR=20,step=1, hrtf=2)